In [79]:
%matplotlib tk

import numpy as np
from typing import List, Optional, Tuple

def plot_multiple_y_axes(
    x: np.ndarray,
    y_data: List[List[np.ndarray]],
    labels: List[List[str]],
    x_label: str = "X-axis",
    y_labels: Optional[List[str]] = None,
    title: Optional[str] = None,
    add_zero_lines: bool = True,
    vertical_lines: Optional[List[Tuple[float, str]]] = None
) -> None:
    """
    Plots multiple y-series on multiple y-axes on a single plot. Each entry in y_data represents an axis,
    and each inner list contains the data series to be plotted on that axis. Similarly, labels is a list of lists
    containing the legend labels for each series on that axis. Additionally, vertical dashed lines can be drawn at
    specified timestamps with vertically rotated labels positioned above the plot.

    Parameters:
    - x (np.ndarray): Data for the x-axis.
    - y_data (List[List[np.ndarray]]): Outer list represents each axis; inner lists contain the data series for that axis.
    - labels (List[List[str]]): Outer list represents each axis; inner lists contain the legend labels for each series.
    - x_label (str, optional): Label for the x-axis. Default is "X-axis".
    - y_labels (List[str], optional): List of y-axis labels, one per axis. If not provided, defaults to "Axis 1", "Axis 2", etc.
    - title (str, optional): Title of the plot.
    - add_zero_lines (bool, optional): Whether to add horizontal zero lines for each axis. Default is True.
    - vertical_lines (List[Tuple[float, str]], optional): List of tuples (timestamp, state_name) where vertical lines
      will be drawn. The state_name is rotated vertically and placed above the plot.

    Returns:
    - None. Displays the plot.
    """
    import matplotlib.pyplot as plt
    from mpl_toolkits.axes_grid1 import host_subplot
    import mpl_toolkits.axisartist as AA
    from matplotlib.transforms import blended_transform_factory

    # --- Parameter validation ---
    if not isinstance(y_data, list):
        raise TypeError("y_data should be a list of lists of y-series.")
    if not isinstance(labels, list):
        raise TypeError("labels should be a list of lists of strings.")
    if len(y_data) != len(labels):
        raise ValueError("The outer lists of y_data and labels must have the same length (one per axis).")
    for i, (axis_data, axis_labels) in enumerate(zip(y_data, labels)):
        if len(axis_data) != len(axis_labels):
            raise ValueError(f"Length mismatch in axis {i}: each data series must have a corresponding label.")

    num_axes = len(y_data)
    if num_axes < 1:
        raise ValueError("At least one axis (y-data list) must be provided.")

    # Set default y-axis labels if not provided
    if y_labels is None:
        y_labels = [f"Axis {i+1}" for i in range(num_axes)]
    if len(y_labels) != num_axes:
        raise ValueError("Length of y_labels must match the number of axes in y_data.")

    # --- Generate Colors Automatically ---
    # Use the default color cycle and assign one color per series (cycling if needed)
    prop_cycle = plt.rcParams['axes.prop_cycle']
    default_colors = prop_cycle.by_key()['color']
    def color_gen():
        idx = 0
        while True:
            yield default_colors[idx % len(default_colors)]
            idx += 1
    cg = color_gen()

    # --- Create Host and Additional Axes ---
    fig = plt.figure(figsize=(10, 6))
    host = host_subplot(111, axes_class=AA.Axes)
    plt.subplots_adjust(right=0.75)

    # Set the x-axis label on the host axis
    host.set_xlabel(x_label)

    # Create additional twin axes for each extra y-axis (beyond the first)
    twin_axes = []
    for i in range(1, num_axes):
        par = host.twinx()
        twin_axes.append(par)

    # Offset the additional axes to avoid label overlap
    offset = 60  # pixels
    for i, par in enumerate(twin_axes):
        new_fixed_axis = par.get_grid_helper().new_fixed_axis
        par.axis["right"] = new_fixed_axis(loc="right", axes=par, offset=(offset * i, 0))
        par.axis["right"].toggle(all=True)
    host.axis["right"].toggle(all=True)

    # --- Plot the Data Series ---
    all_lines = []   # To collect line handles for the combined legend
    all_leg_labels = []  # Corresponding legend labels

    # Plot series on the host axis (axis index 0)
    host_lines = []
    for series, lab in zip(y_data[0], labels[0]):
        c = next(cg)
        line, = host.plot(x, series, color=c, label=lab)
        host_lines.append(line)
        all_lines.append(line)
        all_leg_labels.append(lab)
    host.set_ylabel(y_labels[0])
    if host_lines:
        host.axis["left"].label.set_color(host_lines[0].get_color())

    # Plot series on each additional axis
    for idx, par in enumerate(twin_axes, start=1):
        par_lines = []
        for series, lab in zip(y_data[idx], labels[idx]):
            c = next(cg)
            line, = par.plot(x, series, color=c, label=lab)
            par_lines.append(line)
            all_lines.append(line)
            all_leg_labels.append(lab)
        par.set_ylabel(y_labels[idx])
        if par_lines:
            par.axis["right"].label.set_color(par_lines[0].get_color())
        # Shift the additional y-axis label further right to avoid overlapping
        par.axis["right"].label.set_position(("axes", 1 + 0.1 * (idx - 1)))

    # --- Add Zero Lines ---
    if add_zero_lines:
        # For the host axis, use the color of its first series if available
        host_color = host_lines[0].get_color() if host_lines else 'black'
        host.axhline(0, color=host_color, linestyle='--', linewidth=1)
        for par in twin_axes:
            lines = par.lines
            par_color = lines[0].get_color() if lines else 'black'
            par.axhline(0, color=par_color, linestyle='--', linewidth=1)

    # --- Add Vertical Lines with Vertical (Rotated) Labels Above the Plot ---
    if vertical_lines:
        transform = blended_transform_factory(host.transData, host.transAxes)
        for timestamp, state_name in vertical_lines:
            host.axvline(x=timestamp, color='gray', linestyle='--', linewidth=1)
            host.text(
                timestamp, 1.02, state_name,
                transform=transform,
                rotation=90,
                verticalalignment='bottom',
                horizontalalignment='center',
                fontsize=9,
                bbox=dict(facecolor='white', edgecolor='none', pad=1.0)
            )

    # --- Combined Legend, Grid, and Title ---
    host.legend(all_lines, all_leg_labels, loc='upper left')
    host.grid()
    if title:
        host.set_title(title)

    fig.canvas.draw()
    plt.show()


In [77]:
def pressure_alt(p):
    return 44330 * (1 - (((p) / 1013.25)**( 1 / 5.255)))

def zero_out_columns_before_phase(df, columns, flight_phase_value):
    # Find rows where flight_phase equals the target value
    phase_rows = df[df['flight_phase'] == flight_phase_value]

    if phase_rows.empty:
        print(f"Flight phase '{flight_phase_value}' not found. No changes made.")
        return df

    # Get the timestamp of the first occurrence of the flight phase
    first_phase_time = phase_rows.iloc[0]['timestamp']

    # Zero out the specified columns for all rows with a timestamp earlier than first_phase_time
    df.loc[df['timestamp'] < first_phase_time, columns] = 0

    return df

def plot_columns(df, columns, y_ax_labels, **kwargs):
    plot_multiple_y_axes(
        x=df['timestamp']/1e6,
        y_data=[[df[col_name] for col_name in col_list] for col_list in columns],
        labels=[[col_name for col_name in col_list] for col_list in columns],
        x_label='Time (s)',
        y_labels=y_ax_labels,
        **kwargs
    )


In [62]:
import subprocess
from typing import Optional

def execute_shell_command(command: str) -> Optional[str]:
    """
    Executes a shell command, prints the command and its output.

    Parameters:
    - command (str): The shell command to execute.

    Returns:
    - Optional[str]: The standard output from the command if successful; otherwise, None.
    """
    print(f"Executing Command: {command}\n{'-' * (len('Executing Command: ') + len(command))}")

    try:
        # Execute the command
        result = subprocess.run(
            command,
            shell=True,               # Executes through the shell
            check=True,               # Raises CalledProcessError for non-zero exit codes
            capture_output=True,      # Captures stdout and stderr
            text=True                 # Returns output as string instead of bytes
        )

        # Print standard output
        if result.stdout:
            print("Output:")
            print(result.stdout)

        # Print standard error (if any)
        if result.stderr:
            print("Error Output:")
            print(result.stderr)

        return result.stdout

    except subprocess.CalledProcessError as e:
        # Handle errors in execution
        print(f"Command failed with return code {e.returncode}")
        if e.stdout:
            print("Output:")
            print(e.stdout)
        if e.stderr:
            print("Error Output:")
            print(e.stderr)
        return None
    except FileNotFoundError:
        # Handle the case where the command is not found
        print("Error: Command not found.")
        return None
    except Exception as e:
        # Handle any other exceptions
        print(f"An unexpected error occurred: {e}")
        return None


In [63]:
file_suffix = '2025-02-19-6'

sensor_result = execute_shell_command(rf'py scripts/decode_protobuf_bin.py sensor D:\sensor\dat_{file_suffix}.pb3 sim_out/sensor.csv')
state_result = execute_shell_command(rf'py scripts/decode_protobuf_bin.py state D:\state\fsl_{file_suffix}.pb3 sim_out/state.csv')


Executing Command: py scripts/decode_protobuf_bin.py sensor D:\sensor\dat_2025-02-19-6.pb3 sim_out/sensor.csv
-------------------------------------------------------------------------------------------------------------
Command failed with return code 1
Output:
Copying input file to temporary

Error Output:
Traceback (most recent call last):
  File "c:\Users\imomg\Programming\PSP-HA-Firmware\scripts\decode_protobuf_bin.py", line 124, in <module>
    with open(input_bin_path, 'rb') as original_file:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'D:\\sensor\\dat_2025-02-19-6.pb3'

Executing Command: py scripts/decode_protobuf_bin.py state D:\state\fsl_2025-02-19-6.pb3 sim_out/state.csv
----------------------------------------------------------------------------------------------------------
Command failed with return code 1
Output:
Copying input file to temporary

Error Output:
Traceback (most recent call last):
  File "c:\Users\imomg\Program

In [86]:
import pandas as pd

sensor = pd.read_csv("sim_out/sensor.csv")
state = pd.read_csv("sim_out/state.csv")

combined = pd.merge_asof(
    sensor,
    state,
    on='timestamp',
    direction='nearest',  # Can be 'backward', 'forward', or 'nearest'
)

state_names = [
    "FP_INIT",
    "FP_WAIT",
    "FP_READY",
    "FP_BOOST",
    "FP_COAST",
    "FP_DROGUE",
    "FP_MAIN",
    "FP_LANDED",
    "FP_ERROR",
]

transitions = combined[combined["flight_phase"] != combined["flight_phase"].shift()]
transitions = [
    (transition["timestamp"]/1e6, state_names[int(transition["flight_phase"])])
    for idx, transition in transitions.iterrows()
]

# columns_to_zero = ['pos_vert', 'vel_vert']
# combined = zero_out_columns_before_phase(combined, columns_to_zero, 3)

presalt = pressure_alt(combined['pressure']) - pressure_alt(combined['pressure'].iloc[0])
combined['pos_baro'] = presalt

plot_columns(
    combined,
    [['pos_vert', 'pos_ekf', 'pos_baro'], ['vel_vert', 'vel_ekf']],
    ['Altitude (m)', 'Velocity (m/s)'],
    vertical_lines=transitions
)
